In [1]:
from collections import defaultdict
import numpy as np
from pychunkedgraph.graph import ChunkedGraph
from pychunkedgraph.graph.utils.inspect import dfs_print_node
from pychunkedgraph.graph import cache

from pychunkedgraph.graph.edits import add_edges, remove_edges
from pychunkedgraph.graph.utils.context_managers import TimeIt
from pychunkedgraph.graph.operation import MulticutOperation
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

In [2]:
data = {
    "sources":[["88961898120241790",746048.8125,822299.6875,596280]],
    "sinks":[["88961898120239439",746356.5625,822256.9375,596280]]
}
data = {
    "sources": [["88961898120229847", 746628.5, 822598.5, 596040]],
    "sinks": [["88961966839705767", 746603.4375, 823428.5625, 596040]],
}
data_dict = {}
for k in ["sources", "sinks"]:
    data_dict[k] = defaultdict(list)
    for node in data[k]:
        node_id = node[0]
        x, y, z = node[1:]
        coordinate = np.array([x, y, z]) / cg.meta._ws_cv.resolution
        atomic_id = cg.get_atomic_id_from_coord(
            coordinate[0],
            coordinate[1],
            coordinate[2],
            parent_id=np.uint64(node_id),
        )

        if atomic_id is None:
            raise ValueError("aha")
        data_dict[k]["id"].append(atomic_id)
        data_dict[k]["coord"].append(coordinate)

In [3]:
dfs_print_node(cg, cg.get_root(88961898120229847), limit=2, sep="    ")

 864691138915823844 (12, 1)
     520658338171683838 (7, 4)
         448354453528263012 (6, 3)
             376736664141171760 (5, 1)
                 160949192133443844 (2, 48)
             376741062187767419 (5, 3)
                 304902270963686040 (4, 11)
                     232952978821088701 (3, 1)
                         160879235706127039 (2, 1)
                     232953253698994510 (3, 1)
                         160879304425603610 (2, 1)
                     232812241332733836 (3, 2)
                         160808798242472405 (2, 3)
                         160808866961949083 (2, 67)
                     232953253698994519 (3, 1)
                         160879304425603627 (2, 4)
                     232952978821087494 (3, 1)
                         160879166986650138 (2, 35)
                     232812241332735031 (3, 1)
                         160808866961949502 (2, 1)
                     232812241332735001 (3, 1)
                         160808866961949372 (2, 15)


In [4]:
cache.clear()
cg.cache = cache.CacheService(cg)

op = MulticutOperation(
    cg,
    user_id="test",
    source_ids=data_dict["sources"]["id"],
    sink_ids=data_dict["sinks"]["id"],
    source_coords=data_dict["sources"]["coord"],
    sink_coords=data_dict["sinks"]["coord"],
    bbox_offset=(240,240,24)
)

with TimeIt("remove edges"):
    new_l2_ids = op._apply(operation_id="",timestamp=None)
new_l2_ids

Start remove edges
   Start get_subgraph
   2.0337276458740234
Start run_multicut
   0.01342320442199707

removed edges 68 5
Start get_parents edges
   8.654594421386719e-05
6.043734788894653


IndexError: list index out of range

In [10]:
cg.get_cross_chunk_edges_layer(
    [
        [88961898120229847, 88961966839705767],
        [88961898120241790, 88961898120239439],
    ]
)

array([2, 1])

In [6]:
dfs_print_node(cg, new_l2_ids[0], limit=2, sep="    ")

 864691128455135422 (12, 1)
     233092891675725615 (3, 3)
         161089860902322195 (2, 19)
         161019560877623293 (2, 1)
         161019492158148657 (2, 64)
leaves count 84


In [7]:
dfs_print_node(cg, new_l2_ids[1], limit=2, sep="    ")

 864691128455135423 (12, 1)
     520658338171722206 (7, 4)
         449480353435193961 (6, 1)
             233092891675725616 (3, 2)
                 161019560877623296 (2, 28)
                 161019492158148672 (2, 2)
         449480353435174436 (6, 2)
             377299614094658056 (5, 2)
                 305181546917134562 (4, 1)
                     161230667110155252 (2, 23)
                 305463021893857093 (4, 1)
                     233374366652433123 (3, 2)
                         161300967134856180 (2, 1)
                         161301035854332808 (2, 109)
             377304012141179416 (5, 6)
                 305183745940401547 (4, 1)
                     233093716309443007 (3, 4)
                         161090273219183207 (2, 1)
                         161090273219183061 (2, 168)
                         161019904475005469 (2, 136)
                         161090341938659943 (2, 14)
                 305464121405474998 (4, 1)
                     161301173293286395 